# A journey over APIs for handling geographical data

__Table of contents__<br>
[OpenStreetMap](#OpenStreetMap)<br>
[Nominatim](#Nominatim)<br>
[GeoPy](#GeoPy)<br>
[Overpass API](#Overpass-API)<br>
[Overpass API Python Wrapper](#Overpass-API-Python-Wrapper)<br>
[Geographical Distance](#Geographical-Distance)<br>
[Routing](#Routing)

## OpenStreetMap

http://openstreetmap.org

## Geocoding and reverse geocoding

1. Geocoding: address or place name -> latitude, longitude
2. Reverse geocoding: latitude, longitude -> readable address or place name

## Nominatim

Nominatim https://wiki.openstreetmap.org/wiki/Nominatim 

Note: Nominatim requires each application to provide their own custom user-agent. See their usage policy: https://operations.osmfoundation.org/policies/nominatim/

Since Nominatim is a regular webservice we can query it using `requests` Python package.

In [3]:
import requests

def _query(url, params):
    # required by Nominatim usage policy
    headers = {
        'User-Agent': 'Pyladies Berlin'
    }
    resp = requests.get(url, params, headers=headers)
    if resp.status_code == requests.codes.ok:
        return resp.json()
    resp.raise_for_status()

# geocoding
def nominatim_search(address):
    address = address.replace(' ', '%20')
    url = 'http://nominatim.openstreetmap.org/search/{}'.format(address)
    params = dict(format='jsonv2', addressdetails=1, limit=1)
    return _query(url, params)
    
# reverse geocoding
def nominatim_reverse(lat, lon):
    url = 'http://nominatim.openstreetmap.org/reverse'
    params = dict(format='jsonv2', lat=lat, lon=lon, addressdetails=1, limit=1)
    return _query(url, params)

### Geocoding

In [4]:
import json
some_json = nominatim_search('potsdamer platz, berlin')
print(json.dumps(some_json, indent=2))
    

[
  {
    "place_id": "69085508",
    "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "way",
    "osm_id": "11845212",
    "boundingbox": [
      "52.508628",
      "52.5089126",
      "13.3769102",
      "13.3771842"
    ],
    "lat": "52.5087317",
    "lon": "13.3770476",
    "display_name": "Potsdamer Platz, Mitte, Berlin, 10117, Deutschland",
    "place_rank": "26",
    "category": "highway",
    "type": "secondary",
    "importance": 0.39312074059012,
    "address": {
      "road": "Potsdamer Platz",
      "suburb": "Mitte",
      "city_district": "Mitte",
      "city": "Berlin",
      "state": "Berlin",
      "postcode": "10117",
      "country": "Deutschland",
      "country_code": "de"
    }
  }
]


### Reverse geocoding

In [5]:
lat = '52.5087317'
lon = '13.3770476'

reverse = nominatim_reverse(lat, lon)
print(json.dumps(reverse, indent=2))

{
  "place_id": "3790255",
  "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
  "osm_type": "node",
  "osm_id": "380497986",
  "lat": "52.508862",
  "lon": "13.3771323",
  "place_rank": "30",
  "category": "shop",
  "type": "hairdresser",
  "importance": "0",
  "addresstype": "shop",
  "name": "D. Machts Lounge",
  "display_name": "D. Machts Lounge, Potsdamer Platz, Mitte, Berlin, 10117, Deutschland",
  "address": {
    "hairdresser": "D. Machts Lounge",
    "road": "Potsdamer Platz",
    "suburb": "Mitte",
    "city_district": "Mitte",
    "city": "Berlin",
    "state": "Berlin",
    "postcode": "10117",
    "country": "Deutschland",
    "country_code": "de"
  },
  "boundingbox": [
    "52.508762",
    "52.508962",
    "13.3770323",
    "13.3772323"
  ]
}


## GeoPy

https://geopy.readthedocs.io/en/stable/

### Geocoding

In [6]:
from geopy.geocoders import Nominatim
import json 

def get_geolocator():
    geolocator = Nominatim(user_agent="pyladies-berlin")
    return geolocator

geolocator = get_geolocator()
location = geolocator.geocode(u"Jägerstraße 32, 10117 Berlin", language='en')
print(json.dumps(location.raw, indent=2))

{
  "place_id": "45157935",
  "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
  "osm_type": "node",
  "osm_id": "3213617576",
  "boundingbox": [
    "52.5141169",
    "52.5142169",
    "13.3963086",
    "13.3964086"
  ],
  "lat": "52.5141669",
  "lon": "13.3963586",
  "display_name": "32, J\u00e4gerstra\u00dfe, Spandauer Vorstadt, Mitte, Berlin, 10117, Germany",
  "class": "place",
  "type": "house",
  "importance": 0.44025
}


In [15]:
print(location.address)

32, Jägerstraße, Spandauer Vorstadt, Mitte, Berlin, 10117, Germany


In [7]:
king_latitude = location.latitude
print(king_latitude)

52.5141669


In [8]:
king_longitude = location.longitude
print(king_longitude)

13.3963586


### Reverse geocoding

For reverse geocoding to work with geopy 1.12 we need the hack described in here:
https://github.com/geopy/geopy/issues/262

In [9]:
from geopy.geocoders import Nominatim
from urllib.request import Request

def get_geolocator():
    # Solution from https://github.com/geopy/geopy/issues/262
    geolocator = Nominatim(user_agent="pyladies-berlin")

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

location = get_geolocator().reverse("{}, {}".format(king_latitude, king_longitude))
address = location.address

In [10]:
print(address)

32, Jägerstraße, Spandauer Vorstadt, Mitte, Berlin, 10117, Deutschland


## Overpass API

Overpass API is "a database over the web: the client sends a query to the API and gets back the data set that corresponds to the query."

Below are sample queries you can run on the public Overpass server. 

Go to https://overpass-turbo.eu
1. Search for `Berlin` (or any other place you wish)
2. Copy the queries below (one by one) into the query box on the left and press `Run`. 

![Overpass API how-to](images/overpass-turbo.png "Overpass API how-to")





__Sample queries__

1. Nodes -- Musical instrument shops
```
node
  [shop=musical_instrument]
  ({{bbox}});
out;
```

2. Ways / Areas -- Beaches
```
way
  [natural=beach]
  ({{bbox}});
(._;>;);
out;
```

3. Relations -- Universities
```
relation
  [amenity=university]
  ({{bbox}});
(._;>;);
out;
```


### Search for Ice cream

```
(
  	// our location
	node(52.5141669, 13.3963586, 52.5141669, 13.3963586);
  	// ice cream places in the radius of 1km
	node(around:1000.0, 52.5141669, 13.3963586)["cuisine"="ice_cream"];
    node(around:1000.0, 52.5141669, 13.3963586)["amenity"="ice_cream"];
);
// print results
out body;

// style the ice cream places for better visibility
{{style:
	node[cuisine=ice_cream],
	node[amenity=ice_cream]{
  		icon-image: url('icons/maki/ice-cream-24@2x.png');
  		icon-width: 56;
    }
}}
```


<img src="images/icecream.png" alt="Ice cream search" width="512">

## Defining some utility classes

In [74]:
class IceCream:
    def __init__(self, feature):
        self.lat = feature['geometry']['coordinates'][1]
        self.lon = feature['geometry']['coordinates'][0]
        self.name = feature['properties'].get('name', 'N/A')
        self.coords = (self.lat, self.lon)
        self.geo_dist = float('nan')
        self.road_dist = float('nan')
        self.route = None
        
    def __str__(self):
        return 'Geodesic dist: {: 4.2f} m\tRoad dist: {: 4.2f} m\t{}'.format(self.geo_dist, self.road_dist, self.name)
        


## Overpass API Python Wrapper

In [75]:
import overpass
import json

radius = 1000.0
query = '( node(around:{r},{lat},{lon})["amenity"="ice_cream"]; ' \
          'node(around:{r},{lat},{lon})["cuisine"="ice_cream"]; ' \
        ')'.format(r=radius, lat=king_latitude, lon=king_longitude)
        
api = overpass.API()
response = api.get(query)

features = response['features']
print('Found {} ice cream places'.format(len(features)))

# Uncomment to view the full response
# print(json.dumps(response, indent=2))

Found 5 ice cream places


## Geographical Distance

What is the straight line distance from King to the ice cream place?

Use the [geodesic distance](https://en.wikipedia.org/wiki/Geodesics_on_an_ellipsoid) which is an ellipsoidal-surface formulae
Alternatively one can use the [great circle distance](https://en.wikipedia.org/wiki/Great-circle_distance) which is a spherical surface formulae of the geographic distance

More on geographical distances [Geographical distance](https://en.wikipedia.org/wiki/Geographical_distance)

In [76]:
from geopy import distance

king = (king_latitude, king_longitude)
ice_creams = []

# calculate the geographical distance for each ice cream place
for feature in features:
    ice_cream = IceCream(feature)
    ice_cream.geo_dist = distance.distance(king, ice_cream.coords).m
    ice_creams.append(ice_cream)
    
# sort ice cream according to the geographical distance
ice_creams_geo_dist = sorted(ice_creams, key=lambda x: x.geo_dist)
for ice_cream in ice_creams_geo_dist:
    print(ice_cream)

Geodesic dist:  704.39 m	Road dist:  nan m	Bandy Brooks
Geodesic dist:  751.77 m	Road dist:  nan m	Wonderpots Frozen Yogurt
Geodesic dist:  810.71 m	Road dist:  nan m	Bandy Brooks
Geodesic dist:  820.10 m	Road dist:  nan m	Fedora Eismanufaktur
Geodesic dist:  858.96 m	Road dist:  nan m	kalter Krieg


## Routing

In [77]:
from pyroutelib3 import Router # Import the router
router = Router('foot') # Initialise it

In [78]:
start = router.data.findNode(king_latitude, king_longitude) # Find start and end nodes
print('Start node id in osm is ', start)

Start node id in osm is  29207837


In [79]:
def route_length(route):
    d = 0
    for i in range(len(route)-1):
        d += router.distance(route[i], route[i+1])
    # distance is in km, multiplying by 1000 to get meters
    return d*1000

for ice_cream in ice_creams:
    end = router.data.findNode(ice_cream.lat, ice_cream.lon)
    status, route = router.doRoute(start, end) # Find the route - a list of OSM nodes

    if status == 'success':
        ice_cream.road_dist = route_length(route)
        routeLatLons = list(map(router.nodeLatLon, route)) # Get actual route coordinates
        ice_cream.route = routeLatLons

# order ice_creams according to the road distance
ice_creams_road_dist = sorted(ice_creams, key=lambda x: x.road_dist)
for ice_cream in ice_creams_road_dist:
    print(ice_cream)


Geodesic dist:  704.39 m	Road dist:  1026.64 m	Bandy Brooks
Geodesic dist:  751.77 m	Road dist:  1057.29 m	Wonderpots Frozen Yogurt
Geodesic dist:  820.10 m	Road dist:  1079.49 m	Fedora Eismanufaktur
Geodesic dist:  858.96 m	Road dist:  1168.23 m	kalter Krieg
Geodesic dist:  810.71 m	Road dist:  1169.20 m	Bandy Brooks


Look at the result nb 3, Bandy Brooks. Although it is the 3rd in terms of the geographical distance, it is the furthest in terms of the road distance.

In [85]:
# Copy that to HTML file in this folder to see the map
print('var end=[{}, {}];'.format(ice_creams_road_dist[0].lon, ice_creams_road_dist[0].lat))
print('var start=[{}, {}];'.format(king_longitude, king_latitude))
print('var path={};'.format(ice_creams_road_dist[0].route))

var end=[13.402645, 52.519203];
var start=[13.3963586, 52.5141669];
var path=[[52.5143491, 13.3969553], [52.5144598, 13.3973412], [52.5145683, 13.3977015], [52.5147075, 13.3981526], [52.5148244, 13.3982565], [52.5152812, 13.3979998], [52.5153282, 13.3979735], [52.515411, 13.3979269], [52.5154356, 13.398027], [52.5154572, 13.3981267], [52.5154764, 13.3982154], [52.5155891, 13.3987352], [52.5157998, 13.3995934], [52.515815, 13.3996587], [52.5158835, 13.39993], [52.515901, 13.3999959], [52.5161622, 13.4006294], [52.5161877, 13.4007192], [52.5161916, 13.400797], [52.5161715, 13.4009083], [52.5159727, 13.4011562], [52.5159587, 13.4012545], [52.5159711, 13.4013603], [52.5163583, 13.402266], [52.5164427, 13.4024634], [52.5167195, 13.4031448], [52.5169649, 13.4037369], [52.5169879, 13.4037966], [52.5171978, 13.4043066], [52.5172145, 13.4043458], [52.5172524, 13.4044333], [52.5174453, 13.4043692], [52.5175835, 13.4043357], [52.5179444, 13.4042224], [52.5180278, 13.4041776], [52.5180906, 13.4041